# Checkrail_dspsXfors2_pzdata

In [ ]:
%env PZDATALOC="/Users/dagoret/MacOSX/GitHub/LSST/desc/2024/Delight_cython/rail_dspsXfors2_pz/src/rail/dsps_fors2_pz/data"

In [ ]:
#%env PZDATALOC="/Users/dagoret/MacOSX/GitHub/LSST/desc/2024/Delight_cython/rail_dspsXfors2_pz/src/rail/dsps_fors2_pz/data"

In [ ]:
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from rail.dsps_fors2_pz import readPhotoZHDF5, json_to_inputs, run_from_inputs

In [ ]:
from rail.dsps_fors2_pz import data

In [ ]:
path_data = data.__path__[0]

In [ ]:
ls $path_data

In [ ]:
%env PZDATALOC=$path_data

In [ ]:
PZDATALOC = path_data

In [ ]:
conf_file = os.path.join(path_data,'defaults.json')

input_settings = json_to_inputs(conf_file)
input_settings

In [ ]:
input_settings['photoZ'].keys()

## Filters

In [ ]:
from rail.dsps_fors2_pz  import Observation,sedpyFilter,load_filt,NUV_filt, NIR_filt,get_2lists, load_galaxy

In [ ]:
filters_dict = input_settings['photoZ']['Filters']
filters_dict

In [ ]:
for _f in filters_dict:
    filters_dict[_f]["path"] = os.path.abspath(os.path.join(PZDATALOC, filters_dict[_f]["path"]))
print("Loading filters :")
filters_arr = tuple(sedpyFilter(*load_filt(int(ident), filters_dict[ident]["path"], filters_dict[ident]["transmission"])) for ident in tqdm(filters_dict)) + (NUV_filt, NIR_filt)
N_FILT = len(filters_arr) - 2

In [ ]:
filters_arr

In [ ]:
print("Building templates :")
Xfilt = get_2lists(filters_arr)

In [ ]:
len(Xfilt)

## Data

- If one load the data from `input_settings` retrieve the cosmos2020 data

In [ ]:
from rail.dsps_fors2_pz.io_utils import load_data_for_run

In [ ]:
the_data = load_data_for_run(input_settings)

In [ ]:
zgrid = the_data[0]

In [ ]:
the_sedtemplates = the_data[1]

In [ ]:
the_sednames = list(the_sedtemplates.keys())
the_sednames

In [ ]:
the_sedname = the_sednames[0]
the_sedname

In [ ]:
one_sed = the_sedtemplates[the_sedname] 

In [ ]:
one_sed

## Observations

In [ ]:
print("Loading observations :")
data_path = os.path.abspath(os.path.join(PZDATALOC, input_settings['photoZ']["Dataset"]["path"]))
data_ismag = input_settings['photoZ']["Dataset"]["type"].lower() == "m"

data_file_arr = np.loadtxt(data_path)

In [ ]:
data_file_arr.shape

In [ ]:
plt.hist(data_file_arr[:,2],bins=50)

In [ ]:
obs_arr = []

for i in tqdm(range(data_file_arr.shape[0])):
    try:
        assert (len(data_file_arr[i, :]) == 1 + 2 * N_FILT) or (
                len(data_file_arr[i, :]) == 1 + 2 * N_FILT + 1
            ), f"At least one filter is missing in datapoint {data_file_arr[i,0]} : length is {len(data_file_arr[i,:])}, {1+2*N_FILT} values expected.\nDatapoint removed from dataset."
        # print(int(data_file_arr[i, 0]))
        if len(data_file_arr[i, :]) == 1 + 2 * N_FILT + 1:
            observ = Observation(int(data_file_arr[i, 0]), *load_galaxy(data_file_arr[i, 1 : 2 * N_FILT + 1], data_ismag, id_i_band=input_settings['photoZ']["i_band_num"]), data_file_arr[i, 2 * N_FILT + 1])
        else:
            observ = Observation(int(data_file_arr[i, 0]), *load_galaxy(data_file_arr[i, 1 : 2 * N_FILT + 1], data_ismag, id_i_band=input_settings['photoZ']["i_band_num"]), jnp.nan)
        # print(observ.num)
        obs_arr.extend([observ])
    except AssertionError:
        pass
print("All data loaded and ready for use !")

In [ ]:
len(obs_arr) 

In [ ]:
obs_arr[0]